In [2]:
!pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached afinn-0.1.tar.gz (52 kB)
  Preparing metadata (setup.py) ... done
  Using cached dframcy-0.1.6-py3-none-any.whl (13 kB)
  Using cached lexical_diversity-0.1.1-py3-none-any.whl (117 kB)
  Using cached neurokit2-0.2.7-py2.py3-none-any.whl.metadata (37 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached rpy2-3.5.14.tar.gz (219 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Unable to determine R home: [Errno 2] No such file or directory: 'R'
      cffi mode is CFFI_MODE.ANY
      Looking for R home with: R RHOME
      Unable to determine R home: [Errno 2] No such file or directory: 'R'
      R home found: None
      Error: rpy2 in API mode cannot be built without R in the PATH or R

In [ ]:
import json
from pathlib import Path

from lexical_diversity import lex_div as ld
import neurokit2 as nk
from nltk.tokenize import sent_tokenize
import nltk
import spacy

from utils import *